## Sentiment Analysis - Feedforward NN with n-hot features in DyNet

The following code provides a base implementation of a feedforward neural network in `DyNet` with n-hot features. One difference to `sklearn` is that both features and labels need to be mapped to indices. 

Inspect the code and try to understand the bits and pieces. How are the parameters defined? How is the model created? And how is the model trained?

NB. This is not the quickest code.




In [3]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

ModuleNotFoundError: No module named 'dynet'

In [2]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

def read_dataset(filename):
    """
    Read data and covert to indices
    """
    with open(filename, "r") as f:
        for line in f:
            tag, sentence = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in sentence.split(" ")], t2i[tag])

NameError: name 'defaultdict' is not defined

In [ ]:
def convert_to_n_hot(X, vocab_size):
    """
    Convert instances to n-hot encoding
    :param X:
    :param vocab_size:
    :return:
    """
    out = []
    for word_ids, label in X:
        n_hot = np.zeros(vocab_size)
        for w_idx in word_ids:
            n_hot[w_idx] = 1   # Q1: What happens here?
        out.append((n_hot, label))
    return out

In [ ]:
# Read in the data
train = list(read_dataset("data/classes/train.txt"))

vocab_size = max(w2i.values()) + 1 # OOV
train = convert_to_n_hot(train, vocab_size)
w2i = defaultdict(lambda: UNK, w2i) # freeze vocab
dev = list(read_dataset("data/classes/dev.txt"))
dev = convert_to_n_hot(dev, vocab_size)

ntags = len(t2i)
print(vocab_size, ntags)

In [ ]:
# print the label of the first instance
train[0][1]

In [ ]:
len(train[0][0])

In [ ]:
len(train[1][0])

In [ ]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.SimpleSGDTrainer(model)

In [ ]:
# Define the model
hidden_size = 100
W1 = model.add_parameters((hidden_size, vocab_size))
b1 = model.add_parameters(hidden_size)

W_sm = model.add_parameters((ntags, hidden_size))    # Softmax weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [ ]:
# A function to calculate scores for one value
def calc_scores(input_vec):
    dy.renew_cg()
    n_hot = dy.inputVector(input_vec)
    h = dy.tanh(dy.parameter(W1) * n_hot + dy.parameter(b1))
    score = dy.parameter(W_sm) * h + dy.parameter(b_sm)
    return score

In [ ]:
for ITER in range(20):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
    # Perform testing
    test_correct = 0.0
    for words, tag in dev:
        predict = dy.softmax(calc_scores(words)).npvalue().argmax()
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))